In [ ]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras import layers
from keras import models
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers.convolutional import Conv1D, MaxPooling1D
from tensorflow import keras

In [ ]:
# Montando o drive para conseguir acessar o dataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Carregando o dataset
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/nomes.csv')

In [ ]:
# Chamando o LabelEncoder do Sklearn
le = preprocessing.LabelEncoder()

y = le.fit_transform(df.classification) # Faz um One Hot Encode do sexo(masculino/feminino)
names = df['first_name'].apply(lambda x: x.lower()) # Deixa todos os nomes em letras minúsculas 

In [ ]:
# Cria um vocabulário com todos os caracteres do dataset
vocab = set(' '.join([str(i) for i in names]))            
vocab.add('END')
len_vocab = len(vocab)
char_index = dict((c, i) for i, c in enumerate(vocab))

In [ ]:
maxlen = 14 # Esse é o tamanho do maior nome do dataset

# Essa função seta o valor 1 no vetor, de acordo com a posição da letra e o seu
# valor no vocabulário criado anteriormente
def set_flag(i):
    aux = np.zeros(len_vocab);
    aux[i] = 1
    return list(aux)

# Função que percorre o dataset e chama a funçõa 'set_flag' para setar o valor 1
# na posição correta. Essa funão retor um vetor com todos os nomes codificados.
def prepare_encod_names(X):
    vec_names = []
    trunc_name = [str(i)[0:maxlen] for i in X]  
    for i in trunc_name:
        tmp = [set_flag(char_index[j]) for j in str(i)]
        for k in range(0,maxlen - len(str(i))):
            tmp.append(set_flag(char_index["END"]))
        vec_names.append(tmp)
    return vec_names

In [ ]:
# Codificando os nomes, não recomendo que tente imprimir
x = prepare_encod_names(names.values)

In [ ]:
# Separando os dados em treino, teste e validação 
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
X_train, x_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3)
X_train = np.asarray(X_train)
X_test = np.asarray(X_test)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)
x_val = np.asarray(x_val)
y_val = np.asarray(y_val)

In [ ]:
# Ceiando o modelo de rede neural
# Esse modelo possui as seguintes camadas: Convolução de entrada, concolução oculta,
# achatamento, densa oculta, achatamento e uma densa de saída

cnn = models.Sequential([
                         layers.Conv1D(256, 3, activation='relu', kernel_initializer='he_uniform', input_shape=(14,28)),
                         layers.Conv1D(256, 3, activation='relu'),
                         layers.Flatten(),
                         layers.Dropout(0.2),
                         layers.Dense(100, activation='relu', kernel_initializer='he_uniform'),
                         layers.Dropout(0.2),
                         layers.Dense(1, activation='sigmoid', kernel_initializer='he_uniform')
                         
])

# É possível adicionar um camada de pooling, mas nos meus testes não apresentou melhoria na precisão
# O aumento do tamanho do kernel também não apresentou melhoria, em alguns casos até piorou.
# Por sua vez, a quantidade de filtros é um parâmetro que pode ser alterado visando obter melhorias.

# Não recomendo a alteração das funções de ativação, pois isso resultou em uma drástica queda da precição do modelo
# É possível remover as camadas de dropout sem que suceda um queda significatíva na acurácia do modelo

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9)
opt = keras.optimizers.Adagrad(learning_rate=lr_schedule)
cnn.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

cnn = cnn.fit(X_train, y_train, validation_data=(x_val, y_val),  epochs=100, batch_size=64) # Se quiser salvar o modelo, recomendo que antes de executar essa célula diferncie os nomes do modelo e da história

Epoch 1/100
772/772 [==============================] - 48s 61ms/step - loss: 0.3856 - accuracy: 0.8198 - val_loss: 0.2575 - val_accuracy: 0.8933
Epoch 2/100
772/772 [==============================] - 47s 61ms/step - loss: 0.2531 - accuracy: 0.8931 - val_loss: 0.2235 - val_accuracy: 0.9054
Epoch 3/100
772/772 [==============================] - 47s 61ms/step - loss: 0.2212 - accuracy: 0.9107 - val_loss: 0.1940 - val_accuracy: 0.9212
Epoch 4/100
772/772 [==============================] - 47s 61ms/step - loss: 0.2033 - accuracy: 0.9194 - val_loss: 0.1857 - val_accuracy: 0.9258
Epoch 5/100
772/772 [==============================] - 47s 61ms/step - loss: 0.1894 - accuracy: 0.9261 - val_loss: 0.1802 - val_accuracy: 0.9284
Epoch 6/100
772/772 [==============================] - 47s 61ms/step - loss: 0.1818 - accuracy: 0.9293 - val_loss: 0.1706 - val_accuracy: 0.9333
Epoch 7/100
772/772 [==============================] - 47s 61ms/step - loss: 0.1744 - accuracy: 0.9322 - val_loss: 0.1653 - val_ac